In [ ]:
""Con estos paquetes corre bien el notebook

appdirs==1.4.3
avro==1.8.1
backports-abc==0.5
backports.shutil-get-terminal-size==1.0.0
beautifulsoup4==4.5.3
bleach==2.0.0
brewer2mpl==1.4.1
bs4==0.0.1
cachetools==2.0.0
certifi==2017.4.17
configparser==3.5.0
crcmod==1.7
cycler==0.10.0
datalab==1.0.1
decorator==4.0.11
dill==0.2.6
entrypoints==0.2.2
enum34==1.1.6
funcsigs==1.0.2
functools32==3.2.3.post2
future==0.16.0
futures==3.0.5
gapic-google-logging-v2==0.9.3
gapic-google-pubsub-v1==0.9.3
ggplot==0.6.8
google-api-python-client==1.5.1
google-apitools==0.5.9
google-auth==1.0.0
google-auth-httplib2==0.0.2
google-cloud==0.19.0
google-cloud-bigquery==0.22.1
google-cloud-core==0.22.1
google-cloud-dataflow==0.6.0
google-gax==0.13.0
googleapis-common-protos==1.5.2
googledatastore==7.0.0
grpc-google-logging-v2==0.9.3
grpc-google-pubsub-v1==0.9.3
grpcio==1.2.1
html5lib==0.999999999
httplib2==0.9.2
ipykernel==4.5.2
ipython==5.3.0
ipython-genutils==0.2.0
ipywidgets==6.0.0
Jinja2==2.8
jsonschema==2.6.0
jupyter-client==5.0.1
jupyter-core==4.3.0
MarkupSafe==1.0
matplotlib==1.5.3
mistune==0.7.4
mltoolbox-datalab-classification-and-regression==1.0.0
mltoolbox-datalab-image-classification==0.1
mock==2.0.0
nbconvert==5.1.1
nbformat==4.3.0
nltk==3.2.1
notebook==4.2.3
numpy==1.12.1
oauth2client==2.2.0
packaging==16.8
pandas==0.19.1
pandas-profiling==1.4.0
pandocfilters==1.4.1
pathlib2==2.2.1
patsy==0.4.1
pbr==3.0.1
pexpect==4.2.1
pickleshare==0.7.4
Pillow==3.4.1
plotly==1.12.5
ply==3.8
prompt-toolkit==1.0.14
proto-google-cloud-datastore-v1==0.90.0
protobuf==3.3.0
psutil==4.3.0
ptyprocess==0.5.1
pyasn1==0.2.3
pyasn1-modules==0.0.8
Pygments==2.1.3
pyparsing==2.2.0
python-dateutil==2.5.0
pytz==2016.7
PyYAML==3.11
pyzmq==16.0.2
requests==2.9.1
rsa==3.4.2
scandir==1.5
scikit-learn==0.18.1
scipy==0.18.0
seaborn==0.7.0
simplegeneric==0.8.1
simplejson==3.10.0
singledispatch==3.4.0.3
six==1.10.0
sklearn==0.0
statsmodels==0.6.1
sympy==0.7.6.1
tensorflow==1.1.0
tensorflow-transform==0.1.7
terminado==0.6
testpath==0.3
tornado==4.4.2
traitlets==4.3.2
uritemplate==0.6
wcwidth==0.1.7
webencodings==0.5.1
Werkzeug==0.12.2
widgetsnbextension==2.0.0



""

In [ ]:
%%bash 
pip install sklearn --upgrade
pip install --upgrade tensorflow

In [ ]:
!pip uninstall tensorflow-transform

In [ ]:
# Ejemplo de llamada a la API de BQ

import google.datalab.bigquery as bq

total_births = bq.Query('SELECT CAST(source_year AS string) AS year, COUNT(is_male) AS birth_count FROM `publicdata.samples.natality` GROUP BY year ORDER BY year DESC LIMIT 15')
df = total_births.execute(output_options=bq.QueryOutput.dataframe()).result()
df.head(10)

ax = df.plot(kind='bar',x='year',title='Total births by year')
ax.set_xlabel('Year')
ax.set_ylabel('Birth count')

births_by_weekday = bq.Query('SELECT CAST(wday AS string) AS weekday, SUM(CASE WHEN is_male THEN 1 ELSE 0 END) AS male_births, SUM(CASE WHEN is_male THEN 0 ELSE 1 END) AS female_births FROM `publicdata.samples.natality` WHERE wday IS NOT NULL GROUP BY weekday ORDER BY weekday ASC')
df2 = births_by_weekday.execute(output_options=bq.QueryOutput.dataframe()).result()

df2 = births_by_weekday.execute(output_options=bq.QueryOutput.dataframe()).result()
ax = df2.plot(kind='line',x='weekday',title='Births by weekday')
ax.set_xlabel('Weekday')
ax.set_ylabel('Total')
|

In [ ]:
# Ejemplo de llamada 'mágica' a BQ

import google.datalab.bigquery as bq
import pandas as pd

In [ ]:
%%bq query -n taxi_queries
SELECT trip_seconds, trip_miles, pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude, fare
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE trip_miles IS NOT NULL AND
      trip_seconds IS NOT NULL AND
      pickup_latitude IS NOT NULL AND 
      pickup_longitude IS NOT NULL AND
      dropoff_latitude IS NOT NULL AND
      dropoff_longitude IS NOT NULL AND  
      fare IS NOT NULL
LIMIT 300000



In [ ]:
df = taxi_queries.execute(output_options=bq.QueryOutput.dataframe()).result()

In [ ]:
import numpy as np
import sklearn.model_selection as ms

dataset = df.as_matrix()


X_train, X_test, Y_train, Y_test = ms.train_test_split(dataset[:,:-1], dataset[:,-1], test_size=0.1, random_state=1)

X_train, X_val, Y_train, Y_val = ms.train_test_split(X_train, Y_train, test_size=0.1, random_state=1)

In [137]:
import numpy as np
#import tensorflow
#import tensorflow as tf
import sys
#import tensorflow.contrib.keras as keras

In [138]:
#from tensorflow.contrib.keras import *

In [139]:
from tensorflow.contrib.keras.python.keras.regularizers import l1,l2
from tensorflow.contrib.keras.python.keras.models import Sequential, load_model
from tensorflow.contrib.keras.python.keras.layers import Dense, Dropout, Activation
from tensorflow.contrib.keras.python.keras.constraints import max_norm
from tensorflow.contrib.keras.python.keras.optimizers import RMSprop, Adam
from tensorflow.contrib.keras.python.keras.layers.normalization import BatchNormalization
import os

# Disable info warnings from TF
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'




In [140]:
LIMIT = 200

mask_train = Y_train < 20
mask_val = Y_val < 20
mask_test = Y_test < 20

X_train = X_train[mask_train]
Y_train = Y_train[mask_train]
X_val = X_val[mask_val]
Y_val = Y_val[mask_val]
X_test = X_test[mask_test]
Y_test = Y_test[mask_test]


# Hyperparameters
batch_size = 20
epochs = 10
dropout_rate = 0.5

x_train = X_train[:LIMIT]
y_train = Y_train[:LIMIT]
x_val = X_val[:LIMIT]
y_val = Y_val[:LIMIT]
x_test = X_test[:LIMIT]
y_test = Y_test[:LIMIT]

input_dim = dataset.shape[1] - 1

model = Sequential()
#model.add(Dense(2,input_shape=(input_dim,), init='he_normal'))
model.add(Dense(3,input_shape=(input_dim,)))

model.add(BatchNormalization())
model.add(Activation('elu'))
#model.add(Dropout(dropout_rate))
model.add(Dense(1))

model.summary()

model.compile(loss='mean_squared_error',
              optimizer=Adam(),
              metrics=[keras.metrics.mean_absolute_error, keras.metrics.mean_absolute_percentage_error])


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val))


score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss (mse):', score[0])
print('Test mae:', score[1])
print('Test mape:', score[2])

y_pred = model.predict_proba(x_test, verbose = 0)




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 3)                 21        
_________________________________________________________________
batch_normalization_24 (Batc (None, 3)                 12        
_________________________________________________________________
activation_23 (Activation)   (None, 3)                 0         
_________________________________________________________________
dense_46 (Dense)             (None, 1)                 4         
Total params: 37.0
Trainable params: 31.0
Non-trainable params: 6.0
_________________________________________________________________
Train on 200 samples, validate on 200 samples
Epoch 1/10
200/200 [==============================] - 1s - loss: 86.9629 - mean_absolute_error: 8.2247 - mean_absolute_percentage_error: 96.2434 - val_loss: 405.9774 - val_mean_absolute_error: 19.0914 - val_mean_absolute_perce

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
aux = y_pred[:LIMIT] - 4
ax.scatter(y_test[:LIMIT], aux )
ax.plot([0, 15], [0, 15], 'k--', lw=3)
ax.set_title('Chicago taxi trips prediction')
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
y_pred = model.predict_proba(x_test[50:51], verbose = 0)

In [ ]:
y_pred

In [ ]:
y_test[50:51]

In [141]:
!pip freeze

appdirs==1.4.3
avro==1.8.1
backports-abc==0.5
backports.shutil-get-terminal-size==1.0.0
beautifulsoup4==4.5.3
bleach==2.0.0
brewer2mpl==1.4.1
bs4==0.0.1
cachetools==2.0.0
certifi==2017.4.17
configparser==3.5.0
crcmod==1.7
cycler==0.10.0
datalab==1.0.1
decorator==4.0.11
dill==0.2.6
entrypoints==0.2.2
enum34==1.1.6
funcsigs==1.0.2
functools32==3.2.3.post2
future==0.16.0
futures==3.0.5
gapic-google-logging-v2==0.9.3
gapic-google-pubsub-v1==0.9.3
ggplot==0.6.8
google-api-python-client==1.5.1
google-apitools==0.5.9
google-auth==1.0.0
google-auth-httplib2==0.0.2
google-cloud==0.19.0
google-cloud-bigquery==0.22.1
google-cloud-core==0.22.1
google-cloud-dataflow==0.6.0
google-gax==0.13.0
googleapis-common-protos==1.5.2
googledatastore==7.0.0
grpc-google-logging-v2==0.9.3
grpc-google-pubsub-v1==0.9.3
grpcio==1.2.1
html5lib==0.999999999
httplib2==0.9.2
ipykernel==4.5.2
ipython==5.3.0
ipython-genutils==0.2.0
ipywidgets==6.0.0
Jinja2==2.8
jsonschema==2.6.0
jupyter-client==5.0.1
jupyter-core==4.3.0
